<a href="https://colab.research.google.com/github/VaishnaviNatarajangithub/mediDocAI/blob/main/mediDocAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytesseract pillow spacy transformers torch sentencepiece
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
!pip install easyocr SpeechRecognition langdetect


In [3]:
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [4]:
from PIL import Image
import pytesseract
import spacy
import re
from google.colab import files

nlp = spacy.load("en_core_web_sm")



In [5]:
!pip install easyocr
!pip install opencv-python
!pip install Pillow
!pip install textblob
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [6]:
!apt-get install tesseract-ocr-tam


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-tam is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [7]:
# -----------------------
# 1️⃣ Upload Single Prescription Image
# -----------------------
import os
from google.colab import files
import shutil

# Ensure data folder exists
os.makedirs("data", exist_ok=True)

# Upload one prescription image
uploaded = files.upload()

# Get the uploaded file
filename = list(uploaded.keys())[0]
shutil.move(filename, f"data/{filename}")
image_path = f"data/{filename}"

print("✅ Prescription image uploaded and moved to:", image_path)


Saving prescription_3.png to prescription_3.png
✅ Prescription image uploaded and moved to: data/prescription_3.png


Supports multi language

In [8]:
# Multilingual OCR (English + Tamil as example)
text = pytesseract.image_to_string(Image.open(image_path), lang='eng+tam')
print("----- RAW OCR TEXT -----\n", text)



----- RAW OCR TEXT -----
 Patient: nob Ray
Doctor: Br. vhite

Diagnosis: Flu

Medicine: oseltanivir 75ng 2x daily for 5 days



In [9]:

import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load multilingual NER model
model_name = "Davlan/xlm-roberta-base-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

def extract_entities_multilang(text):
    entities = {
        "patient": None,
        "doctor": None,
        "medicine": None,
        "dosage": None,
        "diagnosis": None,
        "frequency": None,
        "duration": None,
        "allergies": "None"
    }

    # Tamil regex
    tamil_regex = {
        "patient": r"(?:பேசின்ட்|ரோகி பெயர்)[:\-]?\s*([^\n]+)",
        "doctor": r"(?:மருத்துவர்)[:\-]?\s*([^\n]+)",
        "diagnosis": r"(?:பிரதான நோய்கள்)[:\-]?\s*([^\n]+)",
        "medicine": r"(?:மருந்து)[:\-]?\s*([^\d\n]+)",
        "dosage": r"(\d+\s*(mg|g|ml|மி\.கிராம்))",
        "frequency": r"(\d+\s*முறை\s*தினமும்)",
        "duration": r"(\d+\s*நாட்கள்)"
    }
    for key, pattern in tamil_regex.items():
        match = re.search(pattern, text, re.MULTILINE | re.IGNORECASE)
        if match: entities[key] = match.group(1).strip()

    # English regex
    english_regex = {
        "patient": r"Patient[:\-]?\s*([A-Za-z ]+)",
        "doctor": r"Doctor[:\-]?\s*([A-Za-z ]+)",
        "diagnosis": r"(?:Diagnosis|Main Diagnosis)[:\-]?\s*([A-Za-z ]+)",
        "medicine": r"Medicine[:\-]?\s*([A-Za-z]+)",
        "dosage": r"(\d+\s*(mg|g|ml))",
        "frequency": r"(\d+x\s*daily|\d+\s*times daily)",
        "duration": r"for (\d+ days)"
    }
    for key, pattern in english_regex.items():
        match = re.search(pattern, text, re.MULTILINE | re.IGNORECASE)
        if match and not entities[key]: entities[key] = match.group(1).strip()

    # NER fallback
    ner_results = ner_pipeline(text)
    for ent in ner_results:
        word = ent['word'].strip()
        label = ent['entity_group'].upper()
        if not entities['patient'] and ('PATIENT' in label or 'PER' in label): entities['patient'] = word
        if not entities['doctor'] and ('DOCTOR' in label or 'ORG' in label): entities['doctor'] = word
        if not entities['medicine'] and ('MEDICINE' in label or 'MISC' in label): entities['medicine'] = word
        if not entities['dosage'] and any(unit in word.lower() for unit in ['mg','ng','g','ml']): entities['dosage'] = word
        if not entities['diagnosis'] and ('DIAGNOSIS' in label or 'CONDITION' in label): entities['diagnosis'] = word

    return entities

# Extract entities from OCR text
entities = extract_entities_multilang(text)
print("✅ Extracted Entities:")
for key, val in entities.items():
    print(f"{key}: {val}")


Device set to use cpu


✅ Extracted Entities:
patient: nob Ray
doctor: Br
medicine: oseltanivir
dosage: None
diagnosis: Flu
frequency: 2x daily
duration: 5 days
allergies: None


In [10]:
# --- Example ---
ocr_text = """
மருத்துவர்: டாக்டர் ரவி
பேசின்ட்: குமார் ராஜ்
பிரதான நோய்கள்: காய்ச்சல்
மருந்து: பராசிட்டமால் 500மி.கிராம் 3 முறை தினமும் 5 நாட்கள்
"""

entities = extract_entities_multilang(ocr_text)
print("---- Extracted Entities ----")
print(entities)


---- Extracted Entities ----
{'patient': 'குமார் ராஜ்', 'doctor': 'டாக்டர் ரவி', 'medicine': 'பராசிட்டமால்', 'dosage': '500மி.கிராம்', 'diagnosis': 'காய்ச்சல்', 'frequency': '3 முறை தினமும்', 'duration': '5 நாட்கள்', 'allergies': 'None'}


In [11]:
# --- Example ---
ocr_text = """
Doctor: Dr. Ravi
Patient: Kumar Raj
Main Diagnosis: Fever
Medicine: Paracetamol 500mg 3 times daily for 5 days

"""

entities = extract_entities_multilang(ocr_text)
print("---- Extracted Entities ----")
print(entities)


---- Extracted Entities ----
{'patient': 'Kumar Raj', 'doctor': 'Dr', 'medicine': 'Paracetamol', 'dosage': '500mg', 'diagnosis': 'Fever', 'frequency': '3 times daily', 'duration': '5 days', 'allergies': 'None'}


// sample -- with regex


In [ ]:
import re

def extract_entities_regex(text):
    """
    Extract entities dynamically from prescription text (robust version)
    """
    entities = {
        "patient": None,
        "doctor": None,
        "medicine": None,
        "dosage": None,
        "diagnosis": None,
        "frequency": None,
        "duration": None,
        "allergies": "None"
    }

    # Patient: "Patient: Name"
    match = re.search(r"Patient[:\-]?\s*([A-Za-z ]+)", text, re.IGNORECASE)
    if match:
        entities["patient"] = match.group(1).strip()

    # Doctor: "Doctor: Name"
    match = re.search(r"Doctor[:\-]?\s*([A-Za-z\. ]+)", text, re.IGNORECASE)
    if match:
        entities["doctor"] = match.group(1).strip()

    # Diagnosis: "Diagnosis: Flu" (stop at newline)
    match = re.search(r"Diagnosis[:\-]?\s*([^\n]+)", text, re.IGNORECASE)
    if match:
        entities["diagnosis"] = match.group(1).strip()

    # Medicine + Dosage + Frequency + Duration
    match = re.search(
        r"Medicine[:\-]?\s*([\w]+)\s*(\d+\s*(mg|ng|g|ml))?\s*(\d+x\s*daily)?\s*(for \d+ days)?",
        text, re.IGNORECASE
    )
    if match:
        entities["medicine"] = match.group(1).strip()
        entities["dosage"] = match.group(2).strip() if match.group(2) else None
        entities["frequency"] = match.group(4).strip() if match.group(4) else None
        entities["duration"] = match.group(5).strip() if match.group(5) else None

    return entities


In [12]:
!pip install transformers sentencepiece --quiet
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load mBART multilingual model
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def generate_english_summary(ocr_text, entities):
    """
    Generate an English summary from Tamil/English prescription text and extracted entities.
    """
    # Prepare structured text from entities
    input_text = f"Patient: {entities.get('patient','unknown')}. Doctor: {entities.get('doctor','unknown')}. Diagnosis: {entities.get('diagnosis','unknown')}. Medicine: {entities.get('medicine','unknown')} {entities.get('dosage','')} {entities.get('frequency','')} for {entities.get('duration','unknown')}"

    # Set source language as Tamil for mixed text
    tokenizer.src_lang = "ta_IN"
    encoded = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate English summary
    generated_ids = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"],
        max_length=150
    )

    summary = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return summary

'''# --- Example Usage ---
ocr_text = """
மருத்துவர்: டாக்டர் ரவி
பேசின்ட்: குமார் ராஜ்
பிரதான நோய்கள்: காய்ச்சல்
மருந்து: பராசிட்டமால் 500மி.கிராம்
காலம்: 3 நாட்கள்
"""'''

entities = extract_entities_multilang(text)
english_summary = generate_english_summary(text, entities)
print("---- English Summary ----")
print(english_summary)


---- English Summary ----
Diagnosis: Flu. Medicine: oseltanivir None 2x daily for 5 days


In [13]:
ocr_text = """
Doctor: Dr. Ravi
Patient: Kumar Raj
Main Diagnosis: Fever
Medicine: Paracetamol 500mg 3 times daily for 5 days

"""
entities = extract_entities_multilang(ocr_text)
english_summary = generate_english_summary(ocr_text, entities)
print("---- English Summary ----")
print(english_summary)

---- English Summary ----
Diagnosis: Fever. Medicine: Paracetamol 500mg 3 times daily for 5 days


In [14]:
# --- Example Usage ---
ocr_text = """
மருத்துவர்: டாக்டர் ரவி
பேசின்ட்: குமார் ராஜ்
பிரதான நோய்கள்: காய்ச்சல்
மருந்து: பராசிட்டமால் 500மி.கிராம்
காலம்: 3 நாட்கள்
"""
entities = extract_entities_multilang(ocr_text)
english_summary = generate_english_summary(ocr_text, entities)
print("---- English Summary ----")
print(english_summary)

---- English Summary ----
Patient: Kumar Raj. Doctor: Dr. Ravi. Diagnosis: Fever. Medicine: 500mg/day None


In [16]:
!pip uninstall -y whisper


In [17]:
!pip install -U openai-whisper


In [20]:
!pip uninstall -y whisper
!pip install -q openai-whisper
!pip install -q gradio transformers torch



In [24]:
import gradio as gr
import whisper
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# -----------------------
# Load models
# -----------------------

# Whisper speech-to-text
whisper_model = whisper.load_model("base")

# Multilingual QA model
qa_model_name = "deepset/xlm-roberta-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# -----------------------
# Example prescription context
# -----------------------
context_text = """
Doctor: Dr. Ravi
Patient: Kumar Raj
Diagnosis: Fever
Medicine: Paracetamol 500mg
Frequency: 3 times daily
Duration: 5 days
"""

# -----------------------
# Voice QA function
# -----------------------
def voice_qa(audio):
    if not audio:
        return "🎙️ Please record or upload your audio."

    audio_path = audio if isinstance(audio, str) else audio.name
    try:
        result = whisper_model.transcribe(audio_path)
    except Exception as e:
        return f"⚠️ Error processing audio: {e}"

    question = result.get('text', '').strip()

    if not question:
        return "🎙️ No speech detected. Try again."

    try:
        answer = qa_pipeline(question=question, context=context_text)['answer']
    except Exception as e:
        return f"⚠️ Error generating answer: {e}"

    return f"Question: {question}\nAnswer: {answer}"

# -----------------------
# Gradio interface
# -----------------------
iface = gr.Interface(
    fn=voice_qa,
    inputs=gr.Audio(type="filepath", label="🎤 Speak your question"),
    outputs=gr.Textbox(label="💬 Answer"),
    title="Voice-based Multilingual Prescription QA",
    description="🎙️ Ask questions by speaking. The AI will answer based on the prescription context.",
    allow_flagging="never",
    live=False
)

iface.launch()


Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91887c0514a6f8de08.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [25]:
def generate_alerts(entities):
    alerts = []

    # Rule 1: High dosage
    if entities.get("dosage"):
        try:
            dose_val = int(''.join([ch for ch in entities["dosage"] if ch.isdigit()]))
            if dose_val > 1000:
                alerts.append("⚠️ High dosage detected!")
        except:
            pass

    # Rule 2: Long duration
    if entities.get("duration"):
        try:
            duration_val = int(''.join([ch for ch in entities["duration"] if ch.isdigit()]))
            if duration_val > 10:
                alerts.append("⚠️ Long duration, please review prescription!")
        except:
            pass

    # Rule 3: High frequency
    if entities.get("frequency") and any(x in entities["frequency"] for x in ["3", "4"]):
        alerts.append("⚠️ High frequency of medicine intake!")

    # Rule 4: Pediatric dosage warning
    if entities.get("medicine") and "Paracetamol" in entities["medicine"]:
        if "500" in (entities.get("dosage") or ""):
            alerts.append("⚠️ Pediatric dosage warning (check if patient < 12 years).")

    return alerts


# ---------------------------
# Example Test
# ---------------------------
entities = {
    "patient": "Kumar Raj",
    "doctor": "Dr. Ravi",
    "diagnosis": "Fever",
    "medicine": "Paracetamol",
    "dosage": "500 mg",
    "frequency": "3 times daily",
    "duration": "15 days"
}

alerts = generate_alerts(entities)

print("---- Alerts ----")
if alerts:
    for a in alerts:
        print(a)
else:
    print("✅ No alerts. Prescription looks safe.")




---- Alerts ----
⚠️ Long duration, please review prescription!
⚠️ High frequency of medicine intake!
⚠️ Pediatric dosage warning (check if patient < 12 years).
